In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import *
from PIL import Image
from torchvision import transforms
import torchvision.transforms.functional as tf
import os
from torch.optim import RMSprop
from torchvision.utils import make_grid
from pylab import plt
#import Mod.res_block as resblock
import Libm.imporfunc as f_semantic

In [2]:
data_path=os.path.abspath('../../../dataset/GZSL_for_AWA/Animals_with_Attributes2/')
path_model=os.path.abspath('../../../dataset/wiki.en.text.model')
seman_path=os.path.abspath('../')
#vec_model=Word2Vec.load(path_model)#载入语义模型
bat_size=32
img_size=227
gpu=True
tr_epoch=5
worker=2
dim=300

transform1=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])

In [ ]:
#数据集载入
#数据集载入
class SeenData(Dataset):
    def __init__(self,root,transforms=None,train_data=True,label=None):
        #路径
        #self.path=root
        
        if train_data==True:
            dir_class='trainingset'
            #root_label=os.path.abspath('../dataset/Animals_with_Attributes2')
        else:
            dir_class='testingset'
        #seman_dict=np.load(seman_path+'/'+'semantic'+str(dim)+'.npy').item()        
        #label_path=[x.path for x in os.scandir(root) if 'trainclass' in x.path]
        #word=[]
        #for i,file_label in enumerate(label_path):
        #    f=open(file_label,'r')
        #    while True:
        #        temp_w=f.readline().strip()
        #        if not temp_w:
        #             break
        #        word.append(temp_w)
        #    f.close()
        word=f_semantic.get_name_list(root,cls='trainclass')#地址参数有问题
        label_dict=f_semantic.get_dict_name(seman_path)#待修改地址部分参数
        cls_dir,info=f_semantic.get_path_info(word,label_dict,data_path)#地址参数有问题
        self.images_files,label_t,self.label_list,self.label_train=f_semantic.data_pack(cls_dir,info,vec_model)
            
        #path_dir=[]
        #self.label_list=word
        #label_name=[]
        #for key_word in self.label_list:
         #   print(key_word)
         #   [path_dir.append(x.path) for x in os.scandir(root+'/'+dir_class+'/') if x.name.endswith(key_word)]
            #用list存对应的类标名称
            
        #pic_path=[]
        #label_t=[]
        #for l,train_root in enumerate(path_dir):#对训练集文件夹遍历
            #[pic_path.append(x.path) for x in os.scandir(train_root) if x.name.endswith(".jpg")]#每个文件夹里的图片地址
        #    for x in os.scandir(train_root):
        #        if x.name.endswith(".jpg"):
        #            pic_path.append(x.path)
        #            label_t.append(l)
                        
            #pic_path=path_dir#图片文件夹目录
        
        
        label_np=np.array(label_t)
        label_tensor=torch.from_numpy(label_np).type(torch.LongTensor)
        #
        self.images_files=pic_path #图片目录
        self.transforms=transforms
        self.label=label_tensor
        #添加一个返回semantic的功能，在getitem返回 semantic[index,:]
        
    def __getitem__(self,index):#返回一个样本的图片地址，训练类标，语义类标，真实类标
        try:
            pic_data=self.transforms(Image.open(self.images_files[index]))
            
        except RuntimeError:
            print(self.images_files[index])
            return self.transforms(Image.open(self.images_files[index-1])),self.train_label[index],self.seman_label_list[index],self.true_label_list[index]#返回类名
            
        else:
            return pic_data,self.label[index],self.train_label[index],self.seman_label_list[index],self.true_label_list[index]
        
    def __len__(self):
        return len(self.images_files)
        

In [3]:
#数据集载入
class SeenData(Dataset):
    def __init__(self,data_root,seman_dict,transforms=None,data_type='training'):#data_root为总文件夹路径
        if data_type=='training':
            cls_type='trainclasses'
            category='trainingset'
        elif data_type=='zsl':
            cls_type='testclasses'
            category='testingset'
        elif data_type=='gzsl':
            cls_type='gzslclasses'
            category='testingset'
        else:
            print('no model type')
            
        word=f_semantic.get_name_list(data_root,cls_type)#地址为包含类名文件txt的文件夹的地址
        label_dict=f_semantic.get_dict_name(data_root)#
        cls_dir,info=f_semantic.get_path_info(word,label_dict,data_root,category)#是读取训练还是测试集
        self.images_files,label_t,self.seman_list,self.label_train=f_semantic.data_pack(cls_dir,info,seman_dict)
        label_np=np.array(label_t)
        #print(label_t)
        #print(label_np)
        label_tensor=torch.from_numpy(label_np).type(torch.LongTensor)
        #
        #self.images_files=pic_path #图片目录
        self.transforms=transforms
        self.true_label=label_tensor
            
    def __getitem__(self,index):
        try:
            pic_data=self.transforms(Image.open(self.images_files[index]))
            
        except RuntimeError:
            print(self.images_files[index])
            return self.transforms(Image.open(self.images_files[index-1])),self.label_train[index-1],self.seman_list[index-1],self.true_label[index-1]#返回类名
            
        else:
            return pic_data,self.label_train[index],self.seman_list[index],self.true_label[index]
        
    def __len__(self):
        return len(self.images_files)
        
        

In [4]:
#网络
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


In [ ]:
#语义内嵌读取


In [5]:
#载入网络和通用变量
alex=AlexNet(40)
alex.load_state_dict(torch.load('../models/74seenCNNremake.pkl'))
all_seman_dict=np.load(seman_path+'/'+'semantic'+str(dim)+'.npy',allow_pickle=True).item()
indata=SeenData(data_path,all_seman_dict,transform1,data_type='gzsl')#改zsl类型时要改
testingdata=DataLoader(indata,bat_size,shuffle = False,num_workers=worker)

gzslclasses
0
51


In [6]:
#testing  gzsl
#需要准备类标对应的全语义内嵌all_seman
#需要准备训练类对应的语义内嵌semantic_vec
#网络的输出是softmax 最后跟屏蔽低位后跟语义内嵌做积
#if gpu:
#    alex.cuda()
tlabel_name_list=f_semantic.get_dict_name(data_path)

test_label_list=f_semantic.get_name_list(data_path)
seman_temp,_=f_semantic.get_semantic(test_label_list,tlabel_name_list,all_seman_dict)#训练类标对应的semantic
seman_mat=torch.from_numpy(seman_temp)


all_test=f_semantic.get_name_list(data_path,cls='gzslclasses')#zsl的时候testclasses  gzsl为gzslclasses
all_seman,tl_index=f_semantic.get_semantic(all_test,tlabel_name_list,all_seman_dict)#测试用的全部的类标对应的semantic
#all_seman=torch.from_numpy(all_seman_temp)

error_rate=0
total_samp=0#总样本数
error_sam=0
T=10#取前几名
total_cls=40
set_zero=total_cls-T#置零的位数
softm=nn.Softmax(dim=1)
for i_batch,batch in enumerate(testingdata):
    data=batch[0]
    train_label=batch[1]
    semantic_label=batch[2]
    true_label=batch[3]
    true_sample_label=true_label.numpy()
#    if gpu:
#        data=data.cuda()
    output=alex(data)
    soft_out=softm(output)
    #print(output)#缺softmax
    #print(soft_out)
    index_soft=torch.argsort(soft_out,dim=1)
    for i_samp in range(index_soft.size()[0]):
        for i_dim in range(set_zero):#把小的值都屏蔽了
            soft_out[i_samp,index_soft[i_samp,i_dim]]=0
    #print(soft_out)
    result_semanti=torch.matmul(soft_out,seman_mat)#output*semantic_vec#semantic_label是样本类标的semantic，每一列维度个数为训练时类别的个数，应该为bat_size*40  40* 300的两个矩阵
    np_semantic=result_semanti.detach().numpy()
    loc_acc,correct_t,num_s_temp=f_semantic.zsl_el(all_seman,np_semantic,true_sample_label,tl_index,p=1)#结果与全部的semantic做cos距离计算。
    #print(loc_acc)
    error_sam=error_sam+correct_t#计算累计错误样本
    total_samp=total_samp+num_s_temp#总样本个数
    
acc=error_sam/total_samp#正确率
f_acc=1-acc#正确率
print(acc)
print(f_acc)

trainclasses
gzslclasses
0.05308086295204097
0.946919137047959


In [14]:
#testing zsl
print(acc)

1.0
